### Prueba de conexión entre Spark y Mongo, y procesado de los datos con RDDs y Dataframes.

Se cargan los datos desde Mongo con el uso de Pymongo, se genera un RDD a partir de ellos para tratarlos en Spark. 
Generar a partir de este RDD un dataframe para trabajar con el.

Usar el conector oficial entre Mongo y Spark, cargar los datos desde la BBDD como dataframe, y hacer algún preprocesado y limpieza de los datos.

In [1]:
# imports y configuraciones necesarias
import pandas as pd

from pymongo import MongoClient

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, ArrayType

from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, lower, regexp_replace, split
import pyspark.sql.functions as F

import re
import string
import unicodedata

import nltk
from nltk.corpus import stopwords

conf = (SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate())  

sc = SparkContext.getOrCreate(conf=conf)

#### Prueba a obtener los datos desde MongoDB con el uso de Pymongo, y crear un RDD para trabajar en Spark.

In [2]:
# primero establecemos los parámetros de conexión
MONGODB_HOST = 'localhost'
MONGODB_PORT = '27017'
MONGODB_TIMEOUT = 1000
MONGODB_DATABASE = 'tfm_twitter'

URI_CONNECTION = "mongodb://" + MONGODB_HOST + ":" + MONGODB_PORT +  "/"

In [3]:
# ahora realizamos la conexión a la colección que queremos para obtener los tweets y trabajar en Spark ML con ellos
try:
    client = MongoClient(URI_CONNECTION, serverSelectionTimeoutMS=MONGODB_TIMEOUT, maxPoolSize=10)
    client.server_info()
    print('OK -- Connected to MongoDB at server %s' % (MONGODB_HOST))
except pymongo.errors.ServerSelectionTimeoutError as error:
    print('Error with mongoDB connection: %s' % error)
except pymongo.errors.ConnectionFailure as error:
    print('Could not connect to MongoDB: %s' % error)

try:
    destination = 'tweets_spanish'
    collection = client[MONGODB_DATABASE][destination]
    # no nos traemos el _id interno de MongoDB que no debería aportar información que necesitemos
    projection={'_id':0} 
    condition = {}
    result = collection.find(condition,projection)
except Exception as error:
    print("Error getting data: %s" % str(error))

print("El número de registros obtenidos es: ", collection.count_documents(condition))

OK -- Connected to MongoDB at server localhost
El número de registros obtenidos es:  87553


In [4]:
# creamos el rdd a partir de los datos obtenidos desde Mongo
rddjson = sc.parallelize(list(result))

In [5]:
# visualizamos un registro del rdd
rddjson.take(1)

[{'created_at': 'Sun Nov 11 16:34:07 +0000 2018',
  'id': 1.0616583386713784e+18,
  'id_str': '1061658338671378432',
  'text': '@EJFC26 Y Messi',
  'display_text_range': [8.0, 15.0],
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'truncated': False,
  'in_reply_to_status_id': 1.0616582790200566e+18,
  'in_reply_to_status_id_str': '1061658279020056579',
  'in_reply_to_user_id': 1931169114.0,
  'in_reply_to_user_id_str': '1931169114',
  'in_reply_to_screen_name': 'EJFC26',
  'user': {'id': 207220995.0,
   'id_str': '207220995',
   'name': 'Jorge',
   'screen_name': 'jorginho_77',
   'location': 'Desde Gandia hacia el noroeste',
   'url': 'http://tipstertrucho.blogabet.com/',
   'description': 'Economista. Me encanta el fútbol. Culé, mitómano, observador, cabezón e indeciso. Ayrton Senna: El segundo es el primero de los perdedores.',
   'translator_type': 'none',
   'protected': False,
   'verified': False,
   'followers_count': 750.0

#### Prueba a obtener los datos en inglés almacenados en Mongo usando Pymongo y cargarlos a un Dataframe de Pandas, en lugar de cargarlo y probar con un RDD.

In [6]:
# ahora realizamos la conexión a la colección que queremos para obtener los tweets y trabajar en Spark ML con ellos
try:
    client_df = MongoClient(URI_CONNECTION, serverSelectionTimeoutMS=MONGODB_TIMEOUT, maxPoolSize=10)
    client_df.server_info()
    print('OK -- Connected to MongoDB at server %s' % (MONGODB_HOST))
except pymongo.errors.ServerSelectionTimeoutError as error:
    print('Error with mongoDB connection: %s' % error)
except pymongo.errors.ConnectionFailure as error:
    print('Could not connect to MongoDB: %s' % error)

try:
    destination = 'tweets_english'
    collection_df = client_df[MONGODB_DATABASE][destination]
    # no nos traemos el _id interno de MongoDB que no debería aportar información que necesitemos
    projection={'_id':0} 
    condition = {}
    result_df = collection_df.find(condition,projection)
except Exception as error:
    print("Error getting data: %s" % str(error))

print("El número de registros obtenidos es: ", collection_df.count_documents(condition))

OK -- Connected to MongoDB at server localhost
El número de registros obtenidos es:  322364


In [7]:
# pasamos a un dataframe de pandas los datos obtenidos desde MongoDB
df_pandas = pd.DataFrame(list(result_df))

In [8]:
# vistazo a los datos obtenidos
df_pandas.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
0,None,None,Sun Nov 11 16:34:07 +0000 2018,"[8.0, 15.0]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,False,low,...,0.0,0.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@EJFC26 Y Messi,1541954047254,False,"{'id': 207220995.0, 'id_str': '207220995', 'na...",NaN
1,None,None,Fri Mar 22 17:24:18 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1109116195029032961, 'id_str...",NaN,0.0,False,low,...,0.0,0.0,False,{'created_at': 'Fri Mar 22 15:34:45 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @btsmoonchild64: These group photos deserve...,1553275458658,False,"{'id': 864318134, 'id_str': '864318134', 'name...",NaN
2,None,None,Fri Mar 22 17:24:18 +0000 2019,"[15, 27]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,False,low,...,0.0,0.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@rehankkhanNDS Overacting *,1553275458658,False,"{'id': 3304008672, 'id_str': '3304008672', 'na...",NaN
3,None,None,Fri Mar 22 17:24:18 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,False,low,...,0.0,0.0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Pay day play day🤑🤑🤑🤑🤑,1553275458658,False,"{'id': 1091049189235216391, 'id_str': '1091049...",NaN
4,None,None,Fri Mar 22 17:24:18 +0000 2019,"[12, 22]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,False,low,...,0.0,0.0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@pandaeyed1 Thank you!,1553275458658,False,"{'id': 17978056, 'id_str': '17978056', 'name':...",NaN


In [9]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322364 entries, 0 to 322363
Data columns (total 37 columns):
contributors                 0 non-null object
coordinates                  594 non-null object
created_at                   322364 non-null object
display_text_range           70527 non-null object
entities                     322364 non-null object
extended_entities            56062 non-null object
extended_tweet               26830 non-null object
favorite_count               322364 non-null float64
favorited                    322364 non-null bool
filter_level                 322364 non-null object
geo                          594 non-null object
id                           322364 non-null float64
id_str                       322364 non-null object
in_reply_to_screen_name      63503 non-null object
in_reply_to_status_id        60932 non-null float64
in_reply_to_status_id_str    60932 non-null object
in_reply_to_user_id          63503 non-null float64
in_reply_to_user_id_s

In [10]:
# generamos una lista con las columnas que creemos más importantes para posibles análisis a realizar de los tweets.
lista = ['id', 'text', 'extended_tweet', 'entities', 'extended_entities', 'retweet_count',\
    'retweeted_status', 'reply_count', 'quote_count', 'quoted_status', 'favorite_count',\
    'possibly_sensitive', 'source', 'coordinates', 'lang', 'timestamp_ms', 'created_at', 'user']

df_final = df_pandas[lista]
df_final.head()

,id,text,extended_tweet,entities,extended_entities,retweet_count,retweeted_status,reply_count,quote_count,quoted_status,favorite_count,possibly_sensitive,source,coordinates,lang,timestamp_ms,created_at,user
0,1.061658e+18,@EJFC26 Y Messi,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",None,en,1541954047254,Sun Nov 11 16:34:07 +0000 2018,"{'id': 207220995.0, 'id_str': '207220995', 'na..."
1,1.109144e+18,RT @btsmoonchild64: These group photos deserve...,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1109116195029032961, 'id_str...",0.0,{'created_at': 'Fri Mar 22 15:34:45 +0000 2019...,0.0,0.0,NaN,0.0,False,"<a href=""http://twitter.com/download/iphone"" r...",None,en,1553275458658,Fri Mar 22 17:24:18 +0000 2019,"{'id': 864318134, 'id_str': '864318134', 'name..."
2,1.109144e+18,@rehankkhanNDS Overacting *,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,en,1553275458658,Fri Mar 22 17:24:18 +0000 2019,"{'id': 3304008672, 'id_str': '3304008672', 'na..."
3,1.109144e+18,Pay day play day🤑🤑🤑🤑🤑,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",None,en,1553275458658,Fri Mar 22 17:24:18 +0000 2019,"{'id': 1091049189235216391, 'id_str': '1091049..."
4,1.109144e+18,@pandaeyed1 Thank you!,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,en,1553275458658,Fri Mar 22 17:24:18 +0000 2019,"{'id': 17978056, 'id_str': '17978056', 'name':..."


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322364 entries, 0 to 322363
Data columns (total 18 columns):
id                    322364 non-null float64
text                  322364 non-null object
extended_tweet        26830 non-null object
entities              322364 non-null object
extended_entities     56062 non-null object
retweet_count         322364 non-null float64
retweeted_status      188451 non-null object
reply_count           322364 non-null float64
quote_count           322364 non-null float64
quoted_status         50707 non-null object
favorite_count        322364 non-null float64
possibly_sensitive    108250 non-null object
source                322364 non-null object
coordinates           594 non-null object
lang                  322364 non-null object
timestamp_ms          322364 non-null object
created_at            322364 non-null object
user                  322364 non-null object
dtypes: float64(5), object(13)
memory usage: 44.3+ MB


#### Prueba de uso del conector Spark y MongoDB, que carga los datos directamente a un dataframe.

In [12]:
# cargamos desde Mongo los datos en inglés y español mediante el conector con Spark
df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()
df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()

In [13]:
# nos quedamos con el texto de cada DF
df_text_english = df_mongo_english[['text']]
df_text_spanish = df_mongo_spanish[['text']]

# pasamos los DFs a pandas
df_pandas_english = df_text_english.toPandas()
df_pandas_spanish = df_text_spanish.toPandas()

In [14]:
df_pandas_english.head(10)

,text
0,@EJFC26 Y Messi
1,RT @btsmoonchild64: These group photos deserve...
2,@rehankkhanNDS Overacting *
3,Pay day play day🤑🤑🤑🤑🤑
4,@pandaeyed1 Thank you!
5,RT @liamyoung: Strange that Tony Blair has sud...
6,Hard work puts you where good luck can find you.
7,RT @xCiphxr: When creative kids try playing co...
8,RT @bonang_m: I’m working on one as we speak. ...
9,"RT @akashbanerjee: After #PulwamaAttack, terro..."


#### Preprocesado y análisis.

En principio el análisis a realizar será sobre el texto del propio tweet, para intentar ser capaz de clasificarlo como positivo, neutral o negativo en cuanto al sentimiento del tweet.

In [15]:
df_text_english.printSchema()

root
 |-- text: string (nullable = true)



Hay que hacer una limpieza y preprocesado de los textos, para poder analizarlos. Habrá que coger el tweet y eliminar ciertos caracteres como puede ser "RT" que representa que es un retweet, el @usuario:, y otros caracteres y palabras similares que realmente no son parte del texto del tweet para analizar su sentimiento, o no tienen valor para dicho análisis.

In [16]:
# con tokenizer vamos a partir los tweets por palabras
tokenizer = Tokenizer(inputCol = "text", outputCol = "token")

df_tokens_english = tokenizer.transform(df_text_english)
df_tokens_spanish = tokenizer.transform(df_text_spanish)

In [17]:
df_tokens_english.limit(5).toPandas()

,text,token
0,@EJFC26 Y Messi,"[@ejfc26, y, messi]"
1,RT @btsmoonchild64: These group photos deserve...,"[rt, @btsmoonchild64:, these, group, photos, d..."
2,@rehankkhanNDS Overacting *,"[@rehankkhannds, overacting, *]"
3,Pay day play day🤑🤑🤑🤑🤑,"[pay, day, play, day🤑🤑🤑🤑🤑]"
4,@pandaeyed1 Thank you!,"[@pandaeyed1, thank, you!]"


In [18]:
df_tokens_spanish.limit(5).toPandas()

,text,token
0,@EJFC26 Y Messi,"[@ejfc26, y, messi]"
1,RT @MBelenAlegre: Hermoso y sensual viernes ❣️,"[rt, @mbelenalegre:, hermoso, y, sensual, vier..."
2,RT @Foro_TV: Suman 47 muertos y 640 heridos po...,"[rt, @foro_tv:, suman, 47, muertos, y, 640, he..."
3,RT @revistaetcetera: .@lopezobrador_ dice que ...,"[rt, @revistaetcetera:, .@lopezobrador_, dice,..."
4,"Me retracto de mi respuesta , él gobierno se s...","[me, retracto, de, mi, respuesta, ,, él, gobie..."


In [19]:
# función para eliminar palabras que no queramos analizar
def eliminar_stopwords(texto, palabras_eliminar):
    tok = nltk.tokenize
    palabras = tok.word_tokenize(texto)
        
    palabras_salida = []
        
    for palabra in palabras:
        if palabra not in palabras_eliminar:
            palabras_salida.append(palabra)
        
    salida = ""
    for i in range(len(palabras_salida)):
        if palabras_salida[i] in string.punctuation:
            salida = salida.strip()+palabras_salida[i] + " "
        else:
            salida += palabras_salida[i] + " "

    return salida

In [20]:
# cargamos las stopwords por idioma
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

In [21]:
# función de preprocesados de los tweets
def limpieza_tweets_spanish(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 
    # eliminar acentos
    tweet = [''.join((c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')) for s in tweet]
    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, spanish_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)


# función de preprocesados de los tweets en inglés, donde no se aplica la eliminación de acentos y se eliminan
# sus stopwords correspondientes.
def limpieza_tweets_english(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 
    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, english_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)

In [22]:
# limpieza de la columna tokens con las palabras del texto
limpiezaUDF_english = F.udf(limpieza_tweets_english, ArrayType(StringType()))
limpiezaUDF_spanish = F.udf(limpieza_tweets_spanish, ArrayType(StringType()))

df_tokens_english = df_tokens_english.withColumn("tokens_clean", limpiezaUDF_english(df_tokens_english["token"]))
df_tokens_spanish = df_tokens_spanish.withColumn("tokens_clean", limpiezaUDF_spanish(df_tokens_spanish["token"]))

df_tokens_english = df_tokens_english.drop("token")
df_tokens_spanish = df_tokens_spanish.drop("token")

df_tokens_english_clean = df_tokens_english.where(F.size(F.col("tokens_clean")) > 0)
df_tokens_spanish_clean = df_tokens_spanish.where(F.size(F.col("tokens_clean")) > 0)

In [23]:
# visualizamos un grupo de datos para ver el correcto preprocesamiento del texto
df_tokens_english_clean.limit(20).toPandas()

,text,tokens_clean
0,@EJFC26 Y Messi,[messi ]
1,RT @btsmoonchild64: These group photos deserve...,"[group , photos , deserve , attention ]"
2,@rehankkhanNDS Overacting *,[overacting ]
3,Pay day play day🤑🤑🤑🤑🤑,"[pay , day , play , day ]"
4,@pandaeyed1 Thank you!,[thank ]
5,RT @liamyoung: Strange that Tony Blair has sud...,"[strange , tony , blair , suddenly , become , ..."
6,Hard work puts you where good luck can find you.,"[hard , work , puts , good , luck , find ]"
7,RT @xCiphxr: When creative kids try playing co...,"[creative , kids , try , playing , comp ]"
8,RT @bonang_m: I’m working on one as we speak. ...,"[im , working , one , speak , thank , incredib..."
9,"RT @akashbanerjee: After #PulwamaAttack, terro...","[pulwamaattack , terrorists , scored , bigger ..."


Se prueban las conexiones a MongoDB tanto con el uso de la librería PyMongo, como del conector entre Mongo y Spark. También se ve la carga en RDD y en dataframe, para ver distintas opciones de trabajar con los datos.

Se realiza la primera prueba de preprocesado y limpieza de los textos, algo necesario para luego hacer análisis y el cálculo del sentimiento de cada tweet.

Trabajaremos con el conector entre Mongo y Spark cuando queramos usar los datos de Mongo, ya que parece más eficiente y carga directamente los datos en un dataframe, que será la forma con la que seguiremos trabajando los datos, ya que parece lo más eficiente.